In [5]:
import gradio as gr
from graphviz import Digraph
from PIL import Image
import io
import json

# Ocean 테마 설정
theme = gr.themes.Ocean(
    primary_hue="purple",
    secondary_hue="blue",
    neutral_hue="blue",
    text_size="sm",
    spacing_size="sm",
    radius_size="sm"
)

chat_history = []

def handle_input(message, files):
    chat_history.append(("🧑", message))
    response = f"🤖 GPT 분석 결과: \"{message}\" 관련 내용을 확인했어요!"
    chat_history.append(("🤖", response))
    return format_chat(chat_history), "", "예) 사용자님은 글로벌 협업 경험이 있어요! 해당 내용은 기획/PM 직무에 강점이 될 수 있어요."

def format_chat(history):
    return "\n".join([f"{who}: {msg}" for who, msg in history])

def insert_prompt(prompt_text):
    return gr.update(value=prompt_text)

with gr.Blocks(
    theme=theme,
    css="""
    @import url('https://cdn.jsdelivr.net/gh/hangeul-project/neodgm-webfont@latest/neodgm/style.css');

    .gradio-container {
      font-family: 'NeoDunggeunmo', sans-serif !important;
      background-image: url('https://wallpapercrafter.com/desktop/161900-pixel-art-pixelated-pixels-digital-art-space-black-background-universe-planet-stars-Earth-meteors.jpg');
      background-size: cover;
      background-position: center;
      background-repeat: no-repeat;
      background-attachment: fixed;
    }

    .main-container {
      display: flex;
      padding: 30px;
      gap: 30px;
    }

    .left-panel {
      flex: 1;
    }

    .right-panel {
      flex: 2;
      display: flex;
      flex-direction: column;
      gap: 10px;
    }

    .file-box {
      border: 2px dashed #00ffff99;
      padding: 15px;
      border-radius: 10px;
      background-color: #111122cc;
      font-size: 0.7rem;
      margin-bottom: 15px;
    }

    .analysis-box-wrapper {
      position: relative;
      padding-left: 2.5rem;
    }

    .dino-emoji {
      position: absolute;
      left: 0;
      top: -0.5rem;
      font-size: 1.5rem;
    }

    .analysis-box {
      background-color: #ffffffcc;
      color: #000;
      padding: 15px;
      border-radius: 12px;
      font-size: 0.7rem;
      resize: none;
      min-height: 60px;
      overflow-y: auto;
    }

    .chat-box {
      background-color: #0d0d24cc;
      color: #00ffcc;
      padding: 20px;
      border-radius: 12px;
      font-size: 0.75rem;
      height: 350px;
      overflow-y: auto;
      white-space: pre-wrap;
      border: 2px solid #00ffff55;
      box-shadow: 0 0 15px #00ffff55 inset;
    }

    .input-row {
      display: flex;
      gap: 10px;
      margin-top: 10px;
    }

    .chat-input {
      flex: 4;
      background-color: white;
      color: black;
      border-radius: 5px;
      font-size: 0.7rem;
      padding: 10px;
    }

    .send-btn {
      flex: 1;
      background-color: black;
      color: white;
      font-weight: bold;
      border-radius: 5px;
      border: 2px solid white;
    }

    .prompt-buttons {
      display: flex;
      gap: 10px;
      margin-top: 20px;
      justify-content: center;
      flex-wrap: wrap;
    }

    .prompt-button {
      background: #ffffff;
      color: black;
      border-radius: 20px;
      padding: 10px 15px;
      font-size: 0.6rem;
      border: none;
      cursor: pointer;
      font-family: 'NeoDunggeunmo', sans-serif;
      transition: all 0.3s ease-in-out;
    }

    .prompt-button:hover {
      background: linear-gradient(135deg, #6a11cb 0%, #2575fc 100%);
      color: white;
    }
    """
) as demo:

    with gr.Row(elem_classes="main-container"):
        with gr.Column(elem_classes="left-panel", scale=0.5):
            gr.Markdown("📂 파일 등록")
            uploaded = gr.File(file_types=[".pdf", ".docx"], file_count="multiple", elem_classes="file-box")

            with gr.Column(elem_classes="analysis-box-wrapper"):
                gr.Markdown("🦖", elem_classes="dino-emoji")
                analysis_result = gr.Textbox(
                    label="",
                    value="예) 사용자님은 글로벌 협업의 경험이 있으시군요! 해당 내용은 ~~ 직무에 강점이 될 수 있어요.",
                    lines=3,
                    elem_classes="analysis-box",
                    interactive=False
                )

        with gr.Column(elem_classes="right-panel", scale=7.5):
            gr.Markdown("🕹️ RETRO AI CHAT")
            chat_output = gr.Textbox(lines=18, interactive=False, elem_classes="chat-box", label="")
            with gr.Row(elem_classes="input-row"):
                user_input = gr.Textbox(placeholder="Type your message...", elem_classes="chat-input", scale=6.5)
                send_button = gr.Button("SEND", elem_classes="send-btn", scale=0.5)

            with gr.Row(elem_classes="prompt-buttons"):
                btn1 = gr.Button("이력서를 기반으로 직무 추천해줘 >", elem_classes="prompt-button")
                btn2 = gr.Button("나에게 맞춤 채용 공고를 찾아줘 >", elem_classes="prompt-button")
                btn3 = gr.Button("내가 보완해야 할 부분 알려줘 >", elem_classes="prompt-button")
                btn4 = gr.Button("해당 직무의 커리어로드맵을 그려줘 >", elem_classes="prompt-button")

    send_button.click(handle_input, inputs=[user_input, uploaded], outputs=[chat_output, user_input, analysis_result])

    btn1.click(fn=insert_prompt, inputs=[], outputs=[user_input], js="() => '이력서를 기반으로 적합한 직무를 추천해줘'")
    btn2.click(fn=insert_prompt, inputs=[], outputs=[user_input], js="() => '내 이력서와 잘 맞는 채용 공고를 찾아줘'")
    btn3.click(fn=insert_prompt, inputs=[], outputs=[user_input], js="() => '이력서를 보완해야 할 부분이 있다면 알려줘'")
    btn4.click(fn=insert_prompt, inputs=[], outputs=[user_input], js="() => '해당 직무의 커리어로드맵을 그려줘'")

demo.launch(share=True)

c:\Users\wlsth\OneDrive\Desktop\MS AI School 수업자료\2. 2차 프로젝트\.venv\Lib\site-packages\gradio\layouts\column.py:55: UserWarning: 'scale' value should be an integer. Using 0.5 will cause issues.
  warnings.warn(
c:\Users\wlsth\OneDrive\Desktop\MS AI School 수업자료\2. 2차 프로젝트\.venv\Lib\site-packages\gradio\layouts\column.py:55: UserWarning: 'scale' value should be an integer. Using 7.5 will cause issues.
  warnings.warn(
c:\Users\wlsth\OneDrive\Desktop\MS AI School 수업자료\2. 2차 프로젝트\.venv\Lib\site-packages\gradio\components\base.py:203: UserWarning: 'scale' value should be an integer. Using 6.5 will cause issues.
  warnings.warn(
c:\Users\wlsth\OneDrive\Desktop\MS AI School 수업자료\2. 2차 프로젝트\.venv\Lib\site-packages\gradio\components\base.py:203: UserWarning: 'scale' value should be an integer. Using 0.5 will cause issues.
  warnings.warn(
c:\Users\wlsth\OneDrive\Desktop\MS AI School 수업자료\2. 2차 프로젝트\.venv\Lib\site-packages\gradio\utils.py:1024: UserWarning: Expected 1 arguments for function <funct

* Running on local URL:  http://127.0.0.1:7890
* Running on public URL: https://bc20b549e7409fe9b1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
